# BigQuery UDFs for FHIR data

This notebook contains some UDFs to ease querying data stored in BigQuery in the FHIR format. The examples and results below are from the synthetic BCDA example data, but the UDFs should work assuming arguments with the correct type are passed to it.

## Setup

Users of this notebook will need to obtain BigQuery credentials. There are a few ways to do so, but the best option is likely
[this example](https://cloud.google.com/bigquery/docs/authentication/end-user-installed#bigquery-enduser-installed-packages-python). After saving the credentials file, run the following.

In [11]:
#@title Parameters
PROJECT_ID = "lisayin-test" #@param {type:"string"}


In [7]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

import datetime
from google.cloud import bigquery as bq
from google.cloud.bigquery import magics

bqclient = bq.Client(project="betazed")
magics.context.project = PROJECT_ID


Authenticated


## getAllCodesWithSystem

This UDF takes a [codeableConcept](https://www.hl7.org/fhir/datatypes.html#CodeableConcept) (represented in BigQuery with approximately `STRUCT<coding ARRAY<STRUCT<system STRING, version STRING, code STRING, ...>>, text STRING>`) and a system name as a string and returns an array with only the codes and display from codings that match the input system string.

In [10]:
%%bigquery

CREATE TEMP FUNCTION
  getAllCodesWithSystem(codeableConcept ANY TYPE,
    system_check STRING) AS ((
    SELECT
      ARRAY_AGG(STRUCT(c.code,
          c.display)) AS coding
    FROM
      UNNEST(codeableConcept.coding) c
    WHERE
      REGEXP_CONTAINS(c.system, system_check)) );

SELECT
  c.code,
  COUNT(1) AS cnt
FROM (
  SELECT
    getAllCodesWithSystem(type, 'https://bluebutton.cms.gov/resources/codesystem/eob-type') as eob_codes
  FROM `betazed.bcda_synthetic_advanced_large.ExplanationOfBenefit`
), UNNEST(eob_codes) c
GROUP BY c.code
ORDER BY cnt DESC
LIMIT 10;

,code,cnt
0,OUTPATIENT,644890
1,CARRIER,574795
2,PDE,439504
3,INPATIENT,74137
4,HHA,38325
5,SNF,6724
6,HOSPICE,4196


## displayContains

This UDF takes in a [codeableConcept](https://www.hl7.org/fhir/datatypes.html#CodeableConcept) and a string and returns an array of codes where the [coding.display](https://www.hl7.org/fhir/datatypes-definitions.html#Coding.display) contains the input string (case insensitive).

In [12]:
%%bigquery

CREATE TEMP FUNCTION
  displayContains(codeableConcept ANY TYPE,
    name STRING) AS ((
      SELECT
      ARRAY_AGG(STRUCT(c.code,
          c.display)) AS coding
    FROM
        UNNEST(codeableConcept.coding) c
    WHERE
      REGEXP_CONTAINS(UPPER(c.display), UPPER(name))) );

SELECT
  c.display,
  c.code,
  COUNT(1) as cnt
 FROM (
  SELECT
    displayContains(d.diagnosis.codeableConcept, 'diabetes') as contains_results
  FROM `betazed.bcda_synthetic_advanced_large.ExplanationOfBenefit`, UNNEST(diagnosis) d
), UNNEST(contains_results) c
GROUP BY c.code, c.display
ORDER BY cnt DESC
LIMIT 10

,display,code,cnt
0,TYPE 2 DIABETES MELLITUS WITHOUT COMPLICATIONS,E119,312860
1,TYPE 2 DIABETES MELLITUS WITH DIABETIC NEPHROP...,E1121,231209
2,TYPE 2 DIABETES W UNSP DIABETIC RTNOP W/O MACU...,E11319,84638
3,TYPE 2 DIABETES MELLITUS W DIABETIC CHRONIC KI...,E1122,31578
4,TYPE 1 DIABETES MELLITUS W DIABETIC CHRONIC KI...,E1022,26204
5,TYPE 2 DIABETES MELLITUS W OTH DIABETIC KIDNEY...,E1129,20379
6,TYPE 2 DIABETES W UNSP DIABETIC RETINOPATHY W ...,E11311,14132
7,"""TYPE 2 DIABETES MELLITUS WITH DIABETIC NEUROP...",E1140,12593
8,TYPE 2 DIABETES MELLITUS WITH DIABETIC POLYNEU...,E1142,12424
9,TYPE 2 DIABETES W DIABETIC AUTONOMIC (POLY)NEU...,E1143,12267


## codingContains

Similar to the above udf, except it checks the input string against [coding.code](https://www.hl7.org/fhir/datatypes-definitions.html#Coding.code)

In [13]:
%%bigquery

CREATE TEMP FUNCTION
  codingContains(codeableConcept ANY TYPE,
    code_str STRING) AS ((
      SELECT
      ARRAY_AGG(STRUCT(c.code,
          c.display)) AS coding
    FROM
        UNNEST(codeableConcept.coding) c
    WHERE
      REGEXP_CONTAINS(UPPER(c.code), UPPER(code_str))) );

SELECT
  c.display,
  c.code,
  COUNT(1) as cnt
 FROM (
  SELECT
    codingContains(d.diagnosis.codeableConcept, 'E11') as contains_results
  FROM `betazed.bcda_synthetic_advanced_large.ExplanationOfBenefit`, UNNEST(diagnosis) d
), UNNEST(contains_results) c
GROUP BY c.code, c.display
ORDER BY cnt DESC
LIMIT 10

,display,code,cnt
0,TYPE 2 DIABETES MELLITUS WITHOUT COMPLICATIONS,E119,312860
1,TYPE 2 DIABETES MELLITUS WITH DIABETIC NEPHROP...,E1121,231209
2,TYPE 2 DIABETES W UNSP DIABETIC RTNOP W/O MACU...,E11319,84638
3,TYPE 2 DIABETES MELLITUS W DIABETIC CHRONIC KI...,E1122,31578
4,TYPE 2 DIABETES MELLITUS W OTH DIABETIC KIDNEY...,E1129,20379
5,TYPE 2 DIABETES W UNSP DIABETIC RETINOPATHY W ...,E11311,14132
6,"""TYPE 2 DIAB WITH MILD NONP RTNOP WITHOUT MACU...",E113299,13329
7,"""TYPE 2 DIABETES MELLITUS WITH DIABETIC NEUROP...",E1140,12593
8,"""TYPE 2 DIAB WITH SEVERE NONP RTNOP WITHOUT MC...",E113499,12522
9,TYPE 2 DIABETES MELLITUS WITH DIABETIC POLYNEU...,E1142,12424
